In [3]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [11]:
# import the data

data = pd.read_csv("C:\\Users\\rawlo\\Downloads\\credit_card_fraud_2025.csv")
data.head()

,Transaction_ID,Customer_ID,Transaction_Date,Amount,Merchant_Category,Merchant_ID,Card_Type,Transaction_Type,Country,Is_International,Is_Chip,Is_Pin_Used,Distance_From_Home,Hour_of_Day,Device_Type,Fraud_Flag
0,1,25795,2025-05-28 11:54:36,81.53,Online Services,8459,Gold,POS,Germany,1,1,0,1.61,11,Web,0
1,2,10860,2024-09-11 20:26:12,52.19,Fuel,8205,Debit,ATM,Germany,0,1,0,15.77,20,Web,0
2,3,86820,2024-11-02 12:39:23,27.70,Utilities,8076,Gold,Online,USA,0,1,0,9.19,12,Terminal,0
3,4,64886,2024-10-08 21:58:08,9.80,Clothing,3244,Gold,ATM,USA,0,1,0,9.42,21,Web,0
4,5,16265,2024-05-25 20:01:21,178.06,Electronics,3675,Gold,ATM,Germany,0,1,1,1.32,20,Web,0


In [15]:
# check for missing values if any

data.isnull().sum()

Transaction_ID        0
Customer_ID           0
Transaction_Date      0
Amount                0
Merchant_Category     0
Merchant_ID           0
Card_Type             0
Transaction_Type      0
Country               0
Is_International      0
Is_Chip               0
Is_Pin_Used           0
Distance_From_Home    0
Hour_of_Day           0
Device_Type           0
Fraud_Flag            0
dtype: int64

In [23]:
#fill the data if any missing 

data = data.fillna(0)

In [30]:
# check columns
print(data.columns)


Index(['Transaction_ID', 'Customer_ID', 'Transaction_Date', 'Amount',
       'Merchant_Category', 'Merchant_ID', 'Card_Type', 'Transaction_Type',
       'Country', 'Is_International', 'Is_Chip', 'Is_Pin_Used',
       'Distance_From_Home', 'Hour_of_Day', 'Device_Type', 'Fraud_Flag'],
      dtype='object')


In [32]:
data.columns = data.columns.str.strip().str.lower()
print(data.columns)


Index(['transaction_id', 'customer_id', 'transaction_date', 'amount',
       'merchant_category', 'merchant_id', 'card_type', 'transaction_type',
       'country', 'is_international', 'is_chip', 'is_pin_used',
       'distance_from_home', 'hour_of_day', 'device_type', 'fraud_flag'],
      dtype='object')


In [60]:
# Convert categorical columns to numbers
X = pd.get_dummies(X, drop_first=True)

print("Categorical columns converted successfully!")
print(X.head())


Categorical columns converted successfully!
   transaction_id  customer_id  amount  merchant_id  is_international  \
0               1        25795   81.53         8459                 1   
1               2        10860   52.19         8205                 0   
2               3        86820   27.70         8076                 0   
3               4        64886    9.80         3244                 0   
4               5        16265  178.06         3675                 0   

   is_chip  is_pin_used  distance_from_home  hour_of_day  \
0        1            0                1.61           11   
1        1            0               15.77           20   
2        1            0                9.19           12   
3        1            0                9.42           21   
4        1            1                1.32           20   

   transaction_date_year  ...  transaction_type_POS  country_Canada  \
0                   2025  ...                  True           False   
1             

In [66]:

label_column = [col for col in data.columns if 'class' in col or 'fraud' in col or 'target' in col][0]

# Split into features and target
X = data.drop(label_column, axis=1)
y = data[label_column]

# Convert datetime columns
for col in X.columns:
    if 'date' in col or 'time' in col or 'timestamp' in col:
        X[col] = pd.to_datetime(X[col], errors='coerce')
        X[col + '_hour'] = X[col].dt.hour
        X[col + '_day'] = X[col].dt.day
        X[col + '_month'] = X[col].dt.month
        X = X.drop(col, axis=1)

# Convert categorical (string) values → numeric
X = pd.get_dummies(X, drop_first=True)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale numeric features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Data Preprocessing Complete ")


Data Preprocessing Complete 


In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)



Accuracy: 0.984824


C:\Users\rawlo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rawlo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rawlo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    123103
           1       0.00      0.00      0.00      1897

    accuracy                           0.98    125000
   macro avg       0.49      0.50      0.50    125000
weighted avg       0.97      0.98      0.98    125000


Confusion Matrix:
 [[123103      0]
 [  1897      0]]
